# import packages

In [153]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr

from pandas_datareader._utils import RemoteDataError

import warnings

warnings.filterwarnings('ignore')

# import data

In [154]:
data = pd.read_csv("../data/week_1.csv")

#### rewite name as initials

In [155]:
data["name"] = [i.split()[0][0] + i.split()[1][0] for i in data["name"]]

#### sort by name

In [156]:
data = data.sort_values(by=['name','value','ticker'],ascending=True).reset_index(drop=True)

#### calculate initial units purchased at game start date

In [157]:
lookup_date          = '2020-04-24'

buy_prices           = pd.DataFrame()
buy_prices['ticker'] = data[~data['ticker'].isin(['Cash','VAS'])]['ticker'].unique()

buy_prices['price']  = buy_prices.apply(lambda x : pdr.get_data_yahoo(x['ticker'] +".AX").loc[lookup_date]['Close'],axis=1)

In [158]:
#add cash and VAS seperatley due to lookup error
buy_prices_append = pd.DataFrame()

buy_prices_append['ticker'] = ['Cash','VAS']
buy_prices_append['price']  = [1,66.43]

buy_prices = pd.concat([buy_prices,buy_prices_append],axis=0).reset_index(drop=True)

### merge buy prices to initial transactions

In [159]:
data = pd.merge(data,right=buy_prices,how='left',on='ticker')

### calculate units purchased

In [160]:
data['units'] = data.apply(lambda x : np.floor(x['value']/x['price']),axis=1)

### calculate holding value

In [161]:
data['value'] = data.apply(lambda x : np.dot(x['units'],x['price']),axis=1)

### summarise holdings

In [181]:
holdings_total = pd.pivot_table(data=data,index=['name'],values='value',aggfunc=np.sum).reset_index(drop=False).sort_values(by=['value'],ascending=False)

### calculate remaining cash

In [182]:
holdings_total['cash'] = 500000 - holdings_total['value']

### stack holdings and cash

In [190]:
cash_total           = holdings_total
cash_total['ticker'] = 'Cash'
cash_total           = cash_total.rename(columns={'cash':'units'})

final_position = pd.concat([data[['name','ticker','units']],cash_total[['name','ticker','units']]],axis=0).reset_index(drop=True)

### output final holdings

In [201]:
final_position.to_csv("../data/week_1_holdings.csv")